<h1 style='text-align:center'><b> New Data Hotel Booking Cancellation Prediction</b></h1>

## <b><span style='color:darkturquoise'>|</span><span style='color:royalblue'> Importing Library</span></b>

In [1]:
# import libarary 
import os
import pandas as pd
import numpy as np
import pickle
from xgboost.sklearn import XGBClassifier

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

## <b><span style='color:darkturquoise'>|</span><span style='color:royalblue'> Credential Configuration</span></b>

In [2]:
# authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/friskahermalia1/friska-009/sa-development.json"

## <b><span style='color:darkturquoise'>|</span><span style='color:royalblue'> Data Configuration </span></b>

In [3]:
project_id = 'dti-ds'
dataset_id = 'friska_dataset_009'
table_id = 'hotel_cancellation_test'
region = 'us-central1'
bucket_name = 'friska_gcs_009'
model_name = 'model_hotel_booking_cancellation.sav'

## <b><span style='color:darkturquoise'>|</span><span style='color:royalblue'> Test Data Preparation</span></b>

In [4]:
# extreme value data test
new_test_data = pd.DataFrame({
    "market_segment": ["Offline TA/TO", "Groups"],
    "previous_cancellations": [0, 18],
    "reserved_room_type": ["A", "B"],
    "booking_changes": [25, 12],
    "deposit_type": ["Refundable", "Non Refund"],
    "days_in_waiting_list": [20, 2],
    "customer_type": ["Transient-Party", "Transient"],
    "required_car_parking_spaces": [8, 0],
    "total_of_special_requests": [12, 0]
})
new_test_data

,market_segment,previous_cancellations,reserved_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests
0,Offline TA/TO,0,A,25,Refundable,20,Transient-Party,8,12
1,Groups,18,B,12,Non Refund,2,Transient,0,0


In [5]:
# data for bulk prediction
# load data from BQ
client = bigquery.Client(project=project_id)
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
df_test = query_job.result().to_dataframe()
df_test

/home/friskahermalia1/miniconda3/envs/hotelenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,market_segment,previous_cancellations,booking_changes,deposit_type,days_in_waiting_list,customer_type,reserved_room_type,required_car_parking_spaces,total_of_special_requests
0,Direct,0,0,No Deposit,0,Transient-Party,A,0,0
1,Direct,0,1,No Deposit,0,Transient,D,0,0
2,Direct,0,1,No Deposit,0,Transient,A,0,0
3,Direct,0,0,No Deposit,0,Transient,E,1,0
4,Direct,1,1,No Deposit,0,Transient,E,0,0
...,...,...,...,...,...,...,...,...,...
2036,Online TA,4,0,No Deposit,0,Transient,A,0,5
2037,Online TA,0,4,No Deposit,0,Transient,G,1,5
2038,Online TA,0,0,No Deposit,0,Contract,A,0,5
2039,Online TA,0,4,No Deposit,0,Transient,D,0,5


## <b><span style='color:darkturquoise'>|</span><span style='color:royalblue'> Retrieve Model from Google Cloud Storage </span></b>

In [6]:
# Retrieve the model from Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket('friska_gcs_009') # Add bucket name
    blob_model = bucket.blob(f'model/{model_name}')
    blob_model.download_to_filename('model_hotel_booking_cancellation.sav')

    print ("Read model succeeded")
except:
    raise TypeError("An exception occurred")

Read model succeeded


In [7]:
# Load the model
with open('model_hotel_booking_cancellation.sav', 'rb') as f:
    loaded_model = pickle.load(f)

/home/friskahermalia1/miniconda3/envs/hotelenv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.4.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/friskahermalia1/miniconda3/envs/hotelenv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.4.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/friskahermalia1/miniconda3/envs/hotelenv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle 

## <b><span style='color:darkturquoise'>|</span><span style='color:royalblue'> Let's Try to Predict</span></b>

In [8]:
# extreme value data test 
y_pred_cloud = loaded_model.predict(new_test_data)
y_pred_cloud

array([0, 1])

In [9]:
new_test_data_with_predictioan = new_test_data.copy()
new_test_data_with_predictioan['prediction'] = y_pred_cloud
new_test_data_with_predictioan

,market_segment,previous_cancellations,reserved_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,prediction
0,Offline TA/TO,0,A,25,Refundable,20,Transient-Party,8,12,0
1,Groups,18,B,12,Non Refund,2,Transient,0,0,1


In [10]:
# bulk prediction 
y_pred_bulk_cloud = loaded_model.predict(df_test)
df_test_with_prediction = df_test.copy()
df_test_with_prediction['prediction'] = y_pred_bulk_cloud
df_test_with_prediction

,market_segment,previous_cancellations,booking_changes,deposit_type,days_in_waiting_list,customer_type,reserved_room_type,required_car_parking_spaces,total_of_special_requests,prediction
0,Direct,0,0,No Deposit,0,Transient-Party,A,0,0,1
1,Direct,0,1,No Deposit,0,Transient,D,0,0,1
2,Direct,0,1,No Deposit,0,Transient,A,0,0,1
3,Direct,0,0,No Deposit,0,Transient,E,1,0,0
4,Direct,1,1,No Deposit,0,Transient,E,0,0,1
...,...,...,...,...,...,...,...,...,...,...
2036,Online TA,4,0,No Deposit,0,Transient,A,0,5,1
2037,Online TA,0,4,No Deposit,0,Transient,G,1,5,0
2038,Online TA,0,0,No Deposit,0,Contract,A,0,5,0
2039,Online TA,0,4,No Deposit,0,Transient,D,0,5,0


## <b><span style='color:darkturquoise'>|</span><span style='color:royalblue'> Reupload Prediction Result</span></b>

In [11]:
# Name of the destination table in BigQuery
dest_table_id = 'hotel_cancellation_test'
table_full_id = f'{project_id}.{dataset_id}.{dest_table_id}'

# Define schema for the table
schema = [
    bigquery.SchemaField('market_segment', 'STRING'),
    bigquery.SchemaField('previous_cancellations', 'INTEGER'),
    bigquery.SchemaField('booking_changes', 'INTEGER'),
    bigquery.SchemaField('deposit_type', 'STRING'),
    bigquery.SchemaField('days_in_waiting_list', 'INTEGER'),
    bigquery.SchemaField('customer_type', 'STRING'),
    bigquery.SchemaField('reserved_room_type', 'STRING'),
    bigquery.SchemaField('required_car_parking_spaces', 'INTEGER'),
    bigquery.SchemaField('total_of_special_requests', 'INTEGER'),
    bigquery.SchemaField('prediction', 'INTEGER')
]

# Create a reference to the table
table_ref = client.dataset(dataset_id).table(dest_table_id)

# Create job configuration
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE 
)

# Load data into BigQuery table
job = client.load_table_from_dataframe(df_test_with_prediction, table_ref, job_config=job_config)
job.result() 
print(f'Loaded {job.output_rows} rows into {table_full_id}')

Loaded 2041 rows into dti-ds.friska_dataset_009.hotel_cancellation_test
